## Login HF

In [1]:
from huggingface_hub import login
login()


## Transcription using Hinglish Finetuned Whisper from Huggingface

In [ ]:
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
from datasets import load_dataset

import torch
import gc

gc.collect()                      # Python garbage collector
torch.cuda.empty_cache()         # Clears cached memory from PyTorch
torch.cuda.ipc_collect()

device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

model_id = "Oriserve/Whisper-Hindi2Hinglish-Swift"
# model_id = "vasista22/whisper-hindi-large-v2"

model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
)
model.to(device)

processor = AutoProcessor.from_pretrained(model_id)

pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    torch_dtype=torch_dtype,
    device=device,
    return_timestamps=True
)

In [ ]:
result = pipe("/kaggle/input/class-recording/class_recording_2025_03_22.mp3")

with open("/kaggle/working/transcription_output2.txt", "w", encoding="utf-8") as file:
    file.write(result["text"])

print("Transcription saved to transcription_output2.txt")

# print(result["text"])

## Translate Hinglish transcription to English 

In [ ]:
import gc
import torch

# Delete the model and tokenizer (if needed)
del model
del tokenizer

# Empty CUDA cache
torch.cuda.empty_cache()

# Run garbage collector
gc.collect()


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("rudrashah/RLM-hinglish-translator")
model = AutoModelForCausalLM.from_pretrained("rudrashah/RLM-hinglish-translator").to("cuda" if torch.cuda.is_available() else "cpu")
device = model.device

# Template
template = "Hinglish:\n{hi_en}\n\nEnglish:\n"

# Function to chunk transcription
def chunk_transcription(text, tokenizer, max_tokens=30):
    sentences = sent_tokenize(text)
    chunks = []
    current_chunk = ""
    current_token_count = 0

    for sentence in sentences:
        sentence_tokens = len(tokenizer.encode(sentence, add_special_tokens=False))
        if current_token_count + sentence_tokens <= max_tokens:
            current_chunk += " " + sentence
            current_token_count += sentence_tokens
        else:
            chunks.append(current_chunk.strip())
            current_chunk = sentence
            current_token_count = sentence_tokens

    if current_chunk:
        chunks.append(current_chunk.strip())

    return chunks

def extract_english_part(text):
    """
    Extracts and returns the English part from a string containing 'Hinglish:' and 'English:' sections.
    """
    if "English:" in text:
        return text.split("English:", 1)[-1].strip()
    return text.strip()

In [ ]:
from tqdm import tqdm

# Read file and process
with open("/kaggle/input/class-recording/transcription_output2.txt", "r", encoding="utf-8") as f:
    transcription_text = f.read()

chunks = chunk_transcription(transcription_text, tokenizer)

# Initialize a list to store the translated chunks
translated_texts = []

# Translate each chunk with tqdm for progress bar
for i, chunk in tqdm(enumerate(chunks), total=len(chunks), desc="Translating Chunks"):
    prompt = template.format(hi_en=chunk, en="")  # your template likely includes "hi_en: {hi_en} en:"
    input_ids = tokenizer(prompt, return_tensors="pt").to(device)
    output = model.generate(**input_ids, max_new_tokens=350)

    # Decode output and clean special tokens
    translated = tokenizer.decode(output[0], skip_special_tokens=True).strip()

    # If your model generates the output in the form of "en: translated text", clean that
    if "en:" in translated:
        translated = translated.split("en:", 1)[-1].strip()

    translated = extract_english_part(translated)
    translated_texts.append(translated)

# Combine all translated chunks
final_translated_output = "\n".join(translated_texts)

# Save the translated text to a new file
with open("/kaggle/working/translated_lecture_output.txt", "w", encoding="utf-8") as f:
    f.write(final_translated_output)

print("Translation completed and saved to '/kaggle/working/translated_lecture_output.txt'")

In [ ]:
import re

def clean_transcription(text):
    # List of filler/disfluency words (excluding 'right')
    filler_words = ['uhm', 'uh', 'ah', 'like', 'you know', 'so', 'well', 'okay', 'ok', 'hmm', 'huh', 'yeah', 'basically', 'actually']
    
    # Remove basic filler words (case-insensitive)
    pattern = r'\b(?:' + '|'.join(re.escape(word) for word in filler_words) + r')\b'
    cleaned_text = re.sub(pattern, '', text, flags=re.IGNORECASE)

    # Special case: remove 'right?' (with optional space before ?)
    cleaned_text = re.sub(r'\bright\s*\?', '', cleaned_text, flags=re.IGNORECASE)
    
    # Remove extra punctuation clutter like multiple question marks or periods
    cleaned_text = re.sub(r'[?!.]{2,}', '.', cleaned_text)
    
    # Replace multiple spaces or awkward newlines with single space
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text)
    
    # Clean up extra commas or spaces near punctuation
    cleaned_text = re.sub(r'\s+([?.!,])', r'\1', cleaned_text)
    
    return cleaned_text.strip()

# Example usage
with open("/kaggle/input/class-recording/translated_lecture_output (4).txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

cleaned = clean_transcription(raw_text)

with open("/kaggle/working/cleaned_lecture_transcript.txt", "w", encoding="utf-8") as f:
    f.write(cleaned)

print("Done cleaning the transcript!")


In [ ]:
import gc
import torch

# Delete the model and tokenizer (if needed)
del model
del tokenizer

# Empty CUDA cache
torch.cuda.empty_cache()

# Run garbage collector
gc.collect()


## Translate English Transcription to Telugu

In [4]:
!git clone https://github.com/VarunGumma/IndicTransToolkit
# !cd IndicTransToolkit

# !pip install --editable . --use-pep517 # required for pip >= 25.0

# # in case it fails, try:
# # pip install --editable . --use-pep517 --config-settings editable_mode=compat

Cloning into 'IndicTransToolkit'...
remote: Enumerating objects: 232, done.
remote: Counting objects: 100% (137/137), done.
remote: Compressing objects: 100% (78/78), done.
remote: Total 232 (delta 67), reused 102 (delta 49), pack-reused 95 (from 1)
Receiving objects: 100% (232/232), 4.38 MiB | 20.78 MiB/s, done.
Resolving deltas: 100% (95/95), done.


In [ ]:
import torch
import warnings
from tqdm import tqdm
from IndicTransToolkit.processor import IndicProcessor
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

warnings.filterwarnings("ignore")
model_name = "prajdabre/rotary-indictrans2-en-indic-1B"
device = "cuda" if torch.cuda.is_available() else "cpu"

src_lang = "eng_Latn"
tgt_lang = "tel_Telu"
ip = IndicProcessor(inference=True)
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

model = AutoModelForSeq2SeqLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    trust_remote_code=True,
).to(device)


# Load cleaned input file
with open("/kaggle/working/cleaned_lecture_transcript.txt", "r", encoding="utf-8") as f:
    full_text = f.read()

def chunk_text_simple(text, tokenizer, src_lang="eng_Latn", tgt_lang="tel_Telu", max_tokens=400):
    tagged_text = f"{src_lang} {tgt_lang} {text.strip()}"
    input_ids = tokenizer(tagged_text, return_tensors="pt", truncation=False)["input_ids"][0]
    chunks = []

    for i in range(0, len(input_ids), max_tokens):
        chunk_ids = input_ids[i:i + max_tokens]
        chunk_text = tokenizer.decode(chunk_ids, skip_special_tokens=True, clean_up_tokenization_spaces=True)
        chunks.append(chunk_text.strip())

    return chunks


# Chunk the input
chunks = chunk_text_simple(full_text, tokenizer)

print(f"Total chunks created: {len(chunks)}")

# Translate
translated_chunks = []

for i, chunk in tqdm(enumerate(chunks), total=len(chunks), desc="Translating"):
    batch = ip.preprocess_batch([chunk], src_lang=src_lang, tgt_lang=tgt_lang)
    batch = tokenizer(batch, padding="longest", truncation=True, max_length=800, return_tensors="pt").to(device)

    with torch.inference_mode():
        output = model.generate(
            **batch,
            num_beams=10,
            length_penalty=1.5,
            repetition_penalty=2.0,
            num_return_sequences=1,
            max_new_tokens=800,
            early_stopping=True
        )

    decoded = tokenizer.batch_decode(output, skip_special_tokens=True, clean_up_tokenization_spaces=True)
    postprocessed = ip.postprocess_batch(decoded, lang=tgt_lang)
    translated_chunks.append(postprocessed[0])

# Save output
final_translation = "\n\n".join(translated_chunks)

with open("/kaggle/working/translated_lecture_te2.txt", "w", encoding="utf-8") as f:
    f.write(final_translation)

print("Translation complete! Saved to '/kaggle/working/translated_lecture_te2.txt'")

## Sarvam Model

In [ ]:
import traceback

def read_file(file_path, lang_name):
    try:
        with open(file_path, "r", encoding="utf-8") as file:
            # Read the first 5 lines safely
            lines = []
            for _ in range(5):
                try:
                    lines.append(next(file))
                except StopIteration:
                    break

            print(f"=== {lang_name} Text (First Few Lines) ===")
            print("".join(lines))

            # Read the rest
            remaining_text = file.read()

            # Combine and count
            full_doc = "".join(lines) + remaining_text
            total_chars = len(full_doc)
            print(f"\nTotal number of characters in {lang_name} file:", total_chars)

            return full_doc
    except FileNotFoundError:
        print(f"Error: {file_path} not found.")
        return None
    except Exception as e:
        print(f"An error occurred while reading {file_path}: {e}")
        return None


telugu_doc = read_file("/kaggle/input/class-recording/telugu_translation_wp.txt", "Telugu")
print(telugu_doc)

In [2]:
def chunk_text(text, max_length=1000):
    """Splits text into chunks of at most max_length characters while preserving word boundaries."""
    if not text:
        print("Error: Provided text is empty.")
        return []

    chunks = []
    
    while len(text) > max_length:
        split_index = text.rfind(" ", 0, max_length)  # Find the last space within limit
        if split_index == -1:  
            split_index = max_length  # No space found, force split at max_length
        
        chunks.append(text[:split_index].strip())  # Trim spaces before adding
        text = text[split_index:].lstrip()  # Remove leading spaces for the next chunk
    
    if text:
        chunks.append(text.strip())  # Add the last chunk
    
    return chunks


if telugu_doc:
    # Proceed with chunking if the document is valid
    telugu_text_chunks = chunk_text(telugu_doc)
    print(f"Number of chunks created: {len(telugu_text_chunks)}")
else:
    print("Error: Could not read or empty content in the document.")

# Check the first few chunks for debugging purposes
if telugu_text_chunks:
    print("First chunk:")
    print(telugu_text_chunks[:1])  # Print the first 5 chunks
else:
    print("No chunks were created.")


Number of chunks created: 44
First chunk:
['మనం ఇంతకుముందు ఇక్కడ ఉన్నామా? మనం డీప్ లెర్నింగ్ మోడల్స్ గురించి మాట్లాడుకుంటున్నాం కదా? అయితే, గతసారి మనం ఏ విషయం గురించి మాట్లాడుకున్నామో గుర్తుచేసుకోండి? ట్రిప్లెట్ లాస్ చేస్తున్నారు కదా? సరే. అయితే నేను ఒక ఫ్యాక్టర్, ఇంకో ఫ్యాక్టర్ క్యాల్క్యులేట్ చేసి, తర్వాత ఇంకేం ఉంటుంది? దీన్ని పెర్ఫామ్ చేయడానికి మనం రకరకాల డీప్ లెర్నింగ్ మోడల్స్\u200cని ఉపయోగించవచ్చు కదా? సరే. దీన్ని పెర్ఫామ్ చేయడానికి రకరకాల మోడల్స్ ఉంటాయి. ఇప్పుడు ఇది సీ ఎం ఈస్ నెట్\u200cవర్క్ గురించి మనం మాట్లాడుకున్నాం కదా? ఇప్పుడు మనం రెండు విషయాలు చెప్పవచ్చు. మనం సీ ఎం ఈస్ నెట్\u200cవర్క్స్ గురించి మాట్లాడుకుంటున్నాం కదా. ఎందుకంటే, మనం రికగ్నిషన్ గురించి మాట్లాడుకుంటున్నాం? మనం ఏదైనా రికగ్నిషన్ చేస్తున్నప్పుడు, లేదా వెరిఫికేషన్ చేస్తున్నప్పుడు ఏం చేయాలి? మనం ట్రిపుల్ట్ లాస్ చేయవచ్చు లేదా సీ ఎం ఈస్ నెట్\u200cవర్క్\u200cలు కూడా చేయవచ్చు. సరేనా? సీ ఎం వి నెట్\u200cవర్క్స్\u200cలో మేము ఈ షేర్డ్ వెయిట్ సార్ట్ ఆర్కిటెక్చర్నే యూజ్ చేస్తున్నాం. ఇక్కడ రెండు ఇమేజెస్ లేదా రెండు డేటా పాయింట

In [ ]:
import requests

# Define API request details
url = "https://api.sarvam.ai/translate"
headers = {
    "api-subscription-key": "ca70f842-cdc8-4f27-8741-6a1bff54a49b",
    "Content-Type": "application/json"
}

# Initialize dictionary to store translation results
chunk_translation_results = {}

# Send requests for each chunk
for idx, chunk in enumerate(english_text_chunks):
    payload = {
        "source_language_code": "en-IN",
        "target_language_code": "te-IN",
        "speaker_gender": "Female",
        "mode": "classic-colloquial",
        "model": "mayura:v1",
        "enable_preprocessing": False,
        "output_script": "spoken-form-in-native",
        "input": chunk
    }

    try:
        response = requests.post(url, json=payload, headers=headers)
        
        if response.status_code == 200:
            translated_text = response.json().get("translated_text", "Translation not available")
            chunk_translation_results[idx] = {
                "status": "success",
                "original": chunk,
                "translated": translated_text
            }
            print(f"\n=== Translated Chunk {idx + 1} ===\n{translated_text}\n")
        else:
            # If the request fails, store the chunk with an error status
            chunk_translation_results[idx] = {
                "status": "failed",
                "original": chunk,
                "translated": None
            }
            print(f"Error: {response.status_code}, {response.text}")
            print(f"\n=== Failed Chunk {idx + 1} ===\n{chunk}\n")
    
    except Exception as e:
        # In case of exception (network issue, timeout, etc.), store the error
        chunk_translation_results[idx] = {
            "status": "failed",
            "original": chunk,
            "translated": None,
            "error": str(e)
        }
        print(f"Exception occurred for Chunk {idx + 1}: {e}")
        print(f"\n=== Failed Chunk {idx + 1} ===\n{chunk}\n")

# Combine all successfully translated chunks
final_translation = "\n".join([result["translated"] for result in chunk_translation_results.values() if result["status"] == "success"])

# Save successful translations to a text file
with open("/kaggle/working/translated_lecture_telugu.txt", "w", encoding="utf-8") as f:
    f.write(final_translation)

print("Translation saved to '/kaggle/working/translated_lecture_telugu.txt'")

# Optionally, you can also print out the chunk translation results dictionary if you want to check the status of each chunk
print("\n=== Translation Results (Chunk ID: Status) ===")
for chunk_id, result in chunk_translation_results.items():
    print(f"Chunk {chunk_id + 1}: {result['status']}")
    if result['status'] == "failed":
        print(f"Original chunk: {result['original']}")

## Generate Telugu audio in user's voice using reference script

In [2]:
!pip install git+https://github.com/ai4bharat/IndicF5.git

  Cloning https://github.com/ai4bharat/IndicF5.git to /tmp/pip-req-build-2l2lhdo0
  Running command git clone --filter=blob:none --quiet https://github.com/ai4bharat/IndicF5.git /tmp/pip-req-build-2l2lhdo0
  Resolved https://github.com/ai4bharat/IndicF5.git to commit b334bb3b1f6a027420cd02c8ffb1fc95b146c6e8
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.7/88.7 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 84.1 MB/s eta 0:00:00:00:01
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.4 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56

In [5]:
!pip install transformers==4.49.0 pydub soundfile safetensors huggingface_hub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 100.6 MB/s eta 0:00:0000:010:01
  Attempting uninstall: transformers
    Found existing installation: transformers 4.51.1
    Uninstalling transformers-4.51.1:
      Successfully uninstalled transformers-4.51.1


In [6]:
from transformers import AutoModel
import numpy as np
import soundfile as sf

device = "cuda"

# Load IndicF5 from Hugging Face
repo_id = "ai4bharat/IndicF5"
model = AutoModel.from_pretrained(repo_id, trust_remote_code=True).to(device)

telugu_text_chunks = ["మనం ఇంతకుముందు ఇక్కడ ఉన్నామా? మనం డీప్ లెర్నింగ్ మోడల్స్ గురించి మాట్లాడుకుంటున్నాం కదా? అయితే, గతసారి మనం ఏ విషయం గురించి మాట్లాడుకున్నామో గుర్తుచేసుకోండి? ట్రిప్లెట్ లాస్ చేస్తున్నారు కదా? సరే. అయితే నేను ఒక ఫ్యాక్టర్, ఇంకో ఫ్యాక్టర్ క్యాల్క్యులేట్ చేసి, తర్వాత ఇంకేం ఉంటుంది? దీన్ని పెర్ఫామ్ చేయడానికి మనం రకరకాల డీప్ లెర్నింగ్ మోడల్స్\u200cని ఉపయోగించవచ్చు కదా? సరే. దీన్ని పెర్ఫామ్ చేయడానికి రకరకాల మోడల్స్ ఉంటాయి. ఇప్పుడు ఇది సీ ఎం ఈస్ నెట్\u200cవర్క్ గురించి మనం మాట్లాడుకున్నాం కదా? ఇప్పుడు మనం రెండు విషయాలు చెప్పవచ్చు. మనం సీ ఎం ఈస్ నెట్\u200cవర్క్స్ గురించి మాట్లాడుకుంటున్నాం కదా. ఎందుకంటే, మనం రికగ్నిషన్ గురించి మాట్లాడుకుంటున్నాం? మనం ఏదైనా రికగ్నిషన్ చేస్తున్నప్పుడు, లేదా వెరిఫికేషన్ చేస్తున్నప్పుడు ఏం చేయాలి? మనం ట్రిపుల్ట్ లాస్ చేయవచ్చు లేదా సీ ఎం ఈస్ నెట్\u200cవర్క్\u200cలు కూడా చేయవచ్చు. సరేనా? సీ ఎం వి నెట్\u200cవర్క్స్\u200cలో మేము ఈ షేర్డ్ వెయిట్ సార్ట్ ఆర్కిటెక్చర్నే యూజ్ చేస్తున్నాం. ఇక్కడ రెండు ఇమేజెస్ లేదా రెండు డేటా పాయింట్స్ ఉంటాయి. ఉదాహరణకి ఆడియో సిగ్నల్స్. కాబట్టి, రెండు ఆడియో సిగ్నల్స్ పాస్"]

for index, input_text in enumerate(telugu_text_chunks):
    # Now use input_text as input to the model
    audio = model(
        input_text,
        ref_audio_path="/kaggle/input/class-recording/ref_output.wav",
        ref_text="ఈ రెండు డేటా పాయింట్స్ ఒకే క్లాస్‌కి చెందినవా కాదా లేక వేర్వేరు క్లాసెస్‌కి చెందినవా కాదా అని చెప్పడానికి నెట్‌వర్క్‌లో"
    )
    
    # Normalize and save output
    if audio.dtype == np.int16:
        audio = audio.astype(np.float32) / 32768.0
    sf.write(f"/kaggle/working/chunk{index}_output.wav", np.array(audio, dtype=np.float32), samplerate=24000)
    print("Audio saved successfully.")


Download Vocos from huggingface charactr/vocos-mel-24khz


NotImplementedError: Cannot copy out of meta tensor; no data! Please use torch.nn.Module.to_empty() instead of torch.nn.Module.to() when moving module from meta to a different device.